In [304]:
import numpy as np
import pandas as pd
import scipy.io as sio
from sklearn import preprocessing as pre
import scipy.io as sio
from scipy import signal
import math
from scipy.signal import detrend
from scipy.stats import entropy, kurtosis, skew
from sklearn.feature_selection import f_classif, VarianceThreshold
import collections
import scipy.stats as stats

In [148]:
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler, LabelEncoder, FunctionTransformer
from sklearn.compose import ColumnTransformer
import scipy.interpolate as interp
from sklearn.impute import SimpleImputer
import pywt
from PyEMD import EMD 

In [149]:
raw = sio.loadmat('./DREAMER_Dataset/DREAMER.mat')

In [150]:
# create new dataframe with emotion, participant, and video data
a=np.zeros((23,18,9),dtype=object)
for participant in range(0,23):
    for video in range(0,18):
        a[participant,video,0]=raw['DREAMER'][0,0]['Data'][0,participant]['Age'][0][0][0]
        a[participant,video,1]=raw['DREAMER'][0,0]['Data'][0,participant]['Gender'][0][0][0]
        a[participant,video,2]=participant+1
        a[participant,video,3]=video+1
        a[participant,video,4]=['Searching for Bobby Fischer','D.O.A.', 'The Hangover', 'The Ring', '300',
                  'National Lampoon\'s VanWilder', 'Wall-E', 'Crash', 'My Girl', 'The Fly',
                  'Pride and Prejudice', 'Modern Times', 'Remember the Titans', 'Gentlemans Agreement',
                  'Psycho', 'The Bourne Identitiy', 'The Shawshank Redemption', 'The Departed'][video]
        a[participant,video,5]=['calmness', 'surprise', 'amusement', 'fear', 'excitement', 'disgust',
                  'happiness', 'anger', 'sadness', 'disgust', 'calmness', 'amusement',
                  'happiness', 'anger', 'fear', 'excitement', 'sadness', 'surprise'][video]
        a[participant,video,6]=raw['DREAMER'][0,0]['Data'][0,participant]['ScoreValence'][0,0][video,0]
        a[participant,video,7]=raw['DREAMER'][0,0]['Data'][0,participant]['ScoreArousal'][0,0][video,0]
        a[participant,video,8]=raw['DREAMER'][0,0]['Data'][0,participant]['ScoreDominance'][0,0][video,0]
b=pd.DataFrame(a.reshape((23*18,a.shape[2])),columns=['Age','Gender','Participant','Video','Video_Name','Target_Emotion','Valence','Arousal','Dominance'])

In [151]:
b

,Age,Gender,Participant,Video,Video_Name,Target_Emotion,Valence,Arousal,Dominance
0,22,male,1,1,Searching for Bobby Fischer,calmness,4,3,2
1,22,male,1,2,D.O.A.,surprise,3,3,1
2,22,male,1,3,The Hangover,amusement,5,4,4
3,22,male,1,4,The Ring,fear,4,3,2
4,22,male,1,5,300,excitement,4,4,4
...,...,...,...,...,...,...,...,...,...
409,25,male,23,14,Gentlemans Agreement,anger,2,2,2
410,25,male,23,15,Psycho,fear,2,2,2
411,25,male,23,16,The Bourne Identitiy,excitement,3,3,2
412,25,male,23,17,The Shawshank Redemption,sadness,2,2,4


## DWT decomposition

In [153]:
# Function to perform DWT on EEG data and extract frequency bands
def dwt_and_band_extraction(inputAr, eeg_data):
    # Perform DWT with db5 wavelet, 4 levels
    coeffs = pywt.wavedec(eeg_data, 'db5', level=4)
    
    # Extract approximation (cA) and detail coefficients (cD)
    cA4, cD4, cD3, cD2, cD1 = coeffs
    
    # Reconstruct the signals for each band
    theta = cD4
    alpha = cD3
    beta = cD2
    gamma = cD1
    
    inputAr.append(theta)
    inputAr.append(alpha)
    inputAr.append(beta)
    inputAr.append(gamma)
    return inputAr

## Preprocessing

In [154]:
def detrend_signal(signal):
    return detrend(signal, axis=-1)

def baseline_correction(baseline, stimuli):
    baseline_mean = np.mean(baseline, axis=-1, keepdims=True)
    return stimuli - baseline_mean

## EMD decomposition

In [155]:
def find_extrema(signal):
    maxima = (np.diff(np.sign(np.diff(signal))) < 0).nonzero()[0] + 1  # local max
    minima = (np.diff(np.sign(np.diff(signal))) > 0).nonzero()[0] + 1  # local min
    return maxima, minima


def interpolate_extrema(x, signal, extrema):
    t = np.arange(len(signal))
    interpolator = interp.CubicSpline(extrema, signal[extrema])
    return interpolator(t)


def emd(signal, max_imfs=6):
    IMFs = []
    residue = signal
    for _ in range(max_imfs):
        h = residue
        for _ in range(1000):  # Sifting process, with a maximum of 1000 iterations
            maxima, minima = find_extrema(h)
            if len(maxima) < 2 or len(minima) < 2:  # Break if too few extrema
                break
            upper_env = interpolate_extrema(np.arange(len(h)), h, maxima)
            lower_env = interpolate_extrema(np.arange(len(h)), h, minima)
            mean_env = (upper_env + lower_env) / 2
            prev_h = h
            h = h - mean_env

            # Check if h is an IMF
            if np.allclose(prev_h, h, atol=1e-10):
                break

        IMFs.append(h)
        residue = residue - h
        if len(find_extrema(residue)[0]) < 2:
            break
    
    # Ensure exactly 6 IMFs by padding with zeros if necessary
    while len(IMFs) < max_imfs:
        IMFs.append(np.zeros_like(signal))

    return IMFs 

In [156]:
eeg_electrodes = [str(e[0]) for e in raw['DREAMER']['EEG_Electrodes'][0][0][0]]

In [157]:
def feature_extraction_EEG(raw,secs):
    # 128 Hz is the sampling rate for the EEG data
    fs_EEG = 128
    N_EEG = math.ceil(fs_EEG*secs)
    features = []
    for electrode in eeg_electrodes:
        features += [f'{electrode}_theta', f'{electrode}_alpha', f'{electrode}_beta', f'{electrode}_gamma'] + [f'{electrode}_IMF_{i+1}' for i in range(6)]
    # DataFrame to store features
    columns = ['participant', 'video'] + features
    df = pd.DataFrame(columns=columns)
    for participant in range(23):
        for video in range(18):
            feature_values = []
            for i in range(14):
                basl=raw['DREAMER'][0,0]['Data'][0,participant]['EEG'][0,0]['baseline'][0,0][video,0][-1-N_EEG:-1,i]
                stimuli = raw['DREAMER'][0,0]['Data'][0,participant]['EEG'][0,0]['stimuli'][0,0][video,0][-1-N_EEG:-1,i]
                # Detrending
                basl_detrended = detrend_signal(basl)
                stimuli_detrended = detrend_signal(stimuli)
                
                # Baseline correction
                stimuli_corrected = baseline_correction(basl_detrended, stimuli_detrended)
                
                # Feature extraction
                DWT_stimuli = dwt_and_band_extraction([], stimuli_corrected)
                emd_stimuli = emd(stimuli_corrected)

                # Combine features
                feature_values += DWT_stimuli + emd_stimuli
            
            # Append to DataFrame
            row = [participant+1, video+1] + feature_values
            if len(row) == len(columns):
                df.loc[len(df)] = row
            else:
                print(f'Skipping participant {participant+1}, video {video+1}')
                print(len(row))
                print(row)
                print(len(columns))
                print(columns)
    return df

In [ ]:
# Extract features
secs = 5  # Number of seconds for the EEG data segment
initial_df = feature_extraction_EEG(raw, secs)
initial_df.head()

,participant,video,AF3_theta,AF3_alpha,AF3_beta,AF3_gamma,AF3_IMF_1,AF3_IMF_2,AF3_IMF_3,AF3_IMF_4,...,AF4_theta,AF4_alpha,AF4_beta,AF4_gamma,AF4_IMF_1,AF4_IMF_2,AF4_IMF_3,AF4_IMF_4,AF4_IMF_5,AF4_IMF_6
0,1,1,"[-1.8269399327520999, -10.694000738306702, -10...","[0.49736077657430183, 22.400426547369452, 3.30...","[1.4260946835830222, -8.421825879248049, 14.92...","[2.2820572993223274, 2.5997848143652957, -6.51...","[-8.625208967981772, -6.76122618316723, 5.8735...","[271.8306316083523, 239.358160857029, 209.7222...","[-130.83942367373743, -109.05734146942257, -89...","[-547.459771789912, -515.0312470209535, -483.9...",...,"[-5.017270630534736, -109.28545874148115, -36....","[-14.924230842656952, 8.725633821317574, -92.2...","[-11.189911383412282, 30.926470461393173, -11....","[-4.379237005836831, -23.719789559343713, 19.4...","[1.1616431741241842, 22.840090853478113, -21.2...","[-4990.277855528958, -4430.198698333067, -3909...","[4906.51011493665, 4359.930212034154, 3854.331...","[-12557.744211531992, -12287.397192176655, -12...","[16469.13877699183, 16128.517291506201, 15792....","[1518.4948767151295, 1518.4494358108607, 1518...."
1,1,2,"[0.5062581154960033, 0.912007476326554, 2.5752...","[-0.38612141446101633, -6.274040764837486, -2....","[-1.2901555098164261, -1.9465487403542245, -11...","[-2.406950438132705, -1.4401888378845544, 4.42...","[7.187299849210072, 2.48908594819667, -5.76126...","[-68.14040256449726, -56.38957483327619, -46.2...","[90.37696274362476, 77.61107707015923, 66.0474...","[5.638717004127837, 5.69588782145483, 5.703402...",...,"[-5.946356514294608, -47.71219546665877, -35.3...","[-1.4679241714851934, 60.90034386708814, -9.02...","[5.178005664983495, 7.566661770055786, 66.1263...","[12.871990404273404, 4.083632044757508, -12.69...","[389.6261043852249, 288.28589551912705, 225.34...","[-2213.9161111821213, -1879.4657952310267, -15...","[811.67978537938, 709.934075343534, 616.595802...","[741.3340755719194, 660.910481681659, 585.8071...","[-3891.0739228954917, -3828.284381573364, -376...","[-5040.771735148035, -4999.689997758618, -4958..."
2,1,3,"[-0.7417304850594953, -3.4657854661178247, -4....","[0.41278513940362516, 9.861261945260468, 2.318...","[-1.3827799882156016, -18.94342627716695, -10....","[-3.079916736719449, 3.8780490904254394, -1.73...","[-12.81640636829528, -17.285304086989484, -9.2...","[42.225517518158675, 27.278234759306788, 16.16...","[-24.0264830360964, -19.05461680680786, -14.80...","[-7.189789534072112, -5.465502664867119, -3.95...",...,"[-0.7863947527536572, -25.9292939583319, -8.16...","[-4.1270406304651015, -4.402117196897607, -26....","[0.5291522950689149, 38.30107262777281, 25.707...","[7.141072477055207, -3.8397684963850587, 7.067...","[-61.51293984664372, -11.389502357396479, 8.47...","[-1536.4983650468603, -1385.7391196887036, -12...","[871.3828871849325, 748.8048374677428, 637.602...","[1318.3130128562896, 1233.706411411064, 1152.8...","[-877.0838848636753, -834.441221515215, -793.1...","[-6926.798897045539, -6857.590625192434, -6788..."
3,1,4,"[0.22760261879626606, 2.5935007270767776, 1.27...","[0.26103794732345525, -1.3250477622368306, 1.9...","[1.6958236158707125, 10.304412275191863, 11.12...","[2.610260469549624, -2.4266724346828616, 0.671...","[-24.84559520769245, -6.559868828735657, 0.514...","[3.967186796849267, 0.1740433577368301, -2.316...","[65.89434521625127, 55.303895894500045, 45.910...","[-96.5089931246525, -86.61348495962216, -77.40...",...,"[-2.735335694396575, -18.078897014210824, -16....","[-1.7200888836973376, 16.976213858325238, -11....","[-10.009447204338333, -68.64090258333324, -68....","[-18.47044455126826, 2.177489597199492, -2.016...","[64.04700259723303, 0.3136444083759926, -32.98...","[-1985.9300941981808, -1725.659805959508, -148...","[1160.8029499346474, 975.8957382056009, 809.58...","[677.5256820346367, 619.4529066029135, 564.318...","[46.683430700235505, 43.665821605116975, 40.77...","[88.76861021840446, 85.51941443234116, 82.3323..."
4,1,5,"[0.42906

In [ ]:
initial_df

,participant,video,AF3_theta,AF3_alpha,AF3_beta,AF3_gamma,AF3_IMF_1,AF3_IMF_2,AF3_IMF_3,AF3_IMF_4,...,AF4_theta,AF4_alpha,AF4_beta,AF4_gamma,AF4_IMF_1,AF4_IMF_2,AF4_IMF_3,AF4_IMF_4,AF4_IMF_5,AF4_IMF_6
0,1,1,"[-1.8269399327520999, -10.694000738306702, -10...","[0.49736077657430183, 22.400426547369452, 3.30...","[1.4260946835830222, -8.421825879248049, 14.92...","[2.2820572993223274, 2.5997848143652957, -6.51...","[-8.625208967981772, -6.76122618316723, 5.8735...","[271.8306316083523, 239.358160857029, 209.7222...","[-130.83942367373743, -109.05734146942257, -89...","[-547.459771789912, -515.0312470209535, -483.9...",...,"[-5.017270630534736, -109.28545874148115, -36....","[-14.924230842656952, 8.725633821317574, -92.2...","[-11.189911383412282, 30.926470461393173, -11....","[-4.379237005836831, -23.719789559343713, 19.4...","[1.1616431741241842, 22.840090853478113, -21.2...","[-4990.277855528958, -4430.198698333067, -3909...","[4906.51011493665, 4359.930212034154, 3854.331...","[-12557.744211531992, -12287.397192176655, -12...","[16469.13877699183, 16128.517291506201, 15792....","[1518.4948767151295, 1518.4494358108607, 1518...."
1,1,2,"[0.5062581154960033, 0.912007476326554, 2.5752...","[-0.38612141446101633, -6.274040764837486, -2....","[-1.2901555098164261, -1.9465487403542245, -11...","[-2.406950438132705, -1.4401888378845544, 4.42...","[7.187299849210072, 2.48908594819667, -5.76126...","[-68.14040256449726, -56.38957483327619, -46.2...","[90.37696274362476, 77.61107707015923, 66.0474...","[5.638717004127837, 5.69588782145483, 5.703402...",...,"[-5.946356514294608, -47.71219546665877, -35.3...","[-1.4679241714851934, 60.90034386708814, -9.02...","[5.178005664983495, 7.566661770055786, 66.1263...","[12.871990404273404, 4.083632044757508, -12.69...","[389.6261043852249, 288.28589551912705, 225.34...","[-2213.9161111821213, -1879.4657952310267, -15...","[811.67978537938, 709.934075343534, 616.595802...","[741.3340755719194, 660.910481681659, 585.8071...","[-3891.0739228954917, -3828.284381573364, -376...","[-5040.771735148035, -4999.689997758618, -4958..."
2,1,3,"[-0.7417304850594953, -3.4657854661178247, -4....","[0.41278513940362516, 9.861261945260468, 2.318...","[-1.3827799882156016, -18.94342627716695, -10....","[-3.079916736719449, 3.8780490904254394, -1.73...","[-12.81640636829528, -17.285304086989484, -9.2...","[42.225517518158675, 27.278234759306788, 16.16...","[-24.0264830360964, -19.05461680680786, -14.80...","[-7.189789534072112, -5.465502664867119, -3.95...",...,"[-0.7863947527536572, -25.9292939583319, -8.16...","[-4.1270406304651015, -4.402117196897607, -26....","[0.5291522950689149, 38.30107262777281, 25.707...","[7.141072477055207, -3.8397684963850587, 7.067...","[-61.51293984664372, -11.389502357396479, 8.47...","[-1536.4983650468603, -1385.7391196887036, -12...","[871.3828871849325, 748.8048374677428, 637.602...","[1318.3130128562896, 1233.706411411064, 1152.8...","[-877.0838848636753, -834.441221515215, -793.1...","[-6926.798897045539, -6857.590625192434, -6788..."
3,1,4,"[0.22760261879626606, 2.5935007270767776, 1.27...","[0.26103794732345525, -1.3250477622368306, 1.9...","[1.6958236158707125, 10.304412275191863, 11.12...","[2.610260469549624, -2.4266724346828616, 0.671...","[-24.84559520769245, -6.559868828735657, 0.514...","[3.967186796849267, 0.1740433577368301, -2.316...","[65.89434521625127, 55.303895894500045, 45.910...","[-96.5089931246525, -86.61348495962216, -77.40...",...,"[-2.735335694396575, -18.078897014210824, -16....","[-1.7200888836973376, 16.976213858325238, -11....","[-10.009447204338333, -68.64090258333324, -68....","[-18.47044455126826, 2.177489597199492, -2.016...","[64.04700259723303, 0.3136444083759926, -32.98...","[-1985.9300941981808, -1725.659805959508, -148...","[1160.8029499346474, 975.8957382056009, 809.58...","[677.5256820346367, 619.4529066029135, 564.318...","[46.683430700235505, 43.665821605116975, 40.77...","[88.76861021840446, 85.51941443234116, 82.3323..."
4,1,5,"[0.42906

In [ ]:
# Merge the feature DataFrame with metadata DataFrame
merged_df = pd.merge(initial_df, b, left_on=['participant', 'video'], right_on=['Participant', 'Video'])

In [ ]:
merged_df

,participant,video,AF3_theta,AF3_alpha,AF3_beta,AF3_gamma,AF3_IMF_1,AF3_IMF_2,AF3_IMF_3,AF3_IMF_4,...,AF4_IMF_6,Age,Gender,Participant,Video,Video_Name,Target_Emotion,Valence,Arousal,Dominance
0,1,1,"[-1.8269399327520999, -10.694000738306702, -10...","[0.49736077657430183, 22.400426547369452, 3.30...","[1.4260946835830222, -8.421825879248049, 14.92...","[2.2820572993223274, 2.5997848143652957, -6.51...","[-8.625208967981772, -6.76122618316723, 5.8735...","[271.8306316083523, 239.358160857029, 209.7222...","[-130.83942367373743, -109.05734146942257, -89...","[-547.459771789912, -515.0312470209535, -483.9...",...,"[1518.4948767151295, 1518.4494358108607, 1518....",22,male,1,1,Searching for Bobby Fischer,calmness,4,3,2
1,1,2,"[0.5062581154960033, 0.912007476326554, 2.5752...","[-0.38612141446101633, -6.274040764837486, -2....","[-1.2901555098164261, -1.9465487403542245, -11...","[-2.406950438132705, -1.4401888378845544, 4.42...","[7.187299849210072, 2.48908594819667, -5.76126...","[-68.14040256449726, -56.38957483327619, -46.2...","[90.37696274362476, 77.61107707015923, 66.0474...","[5.638717004127837, 5.69588782145483, 5.703402...",...,"[-5040.771735148035, -4999.689997758618, -4958...",22,male,1,2,D.O.A.,surprise,3,3,1
2,1,3,"[-0.7417304850594953, -3.4657854661178247, -4....","[0.41278513940362516, 9.861261945260468, 2.318...","[-1.3827799882156016, -18.94342627716695, -10....","[-3.079916736719449, 3.8780490904254394, -1.73...","[-12.81640636829528, -17.285304086989484, -9.2...","[42.225517518158675, 27.278234759306788, 16.16...","[-24.0264830360964, -19.05461680680786, -14.80...","[-7.189789534072112, -5.465502664867119, -3.95...",...,"[-6926.798897045539, -6857.590625192434, -6788...",22,male,1,3,The Hangover,amusement,5,4,4
3,1,4,"[0.22760261879626606, 2.5935007270767776, 1.27...","[0.26103794732345525, -1.3250477622368306, 1.9...","[1.6958236158707125, 10.304412275191863, 11.12...","[2.610260469549624, -2.4266724346828616, 0.671...","[-24.84559520769245, -6.559868828735657, 0.514...","[3.967186796849267, 0.1740433577368301, -2.316...","[65.89434521625127, 55.303895894500045, 45.910...","[-96.5089931246525, -86.61348495962216, -77.40...",...,"[88.76861021840446, 85.51941443234116, 82.3323...",22,male,1,4,The Ring,fear,4,3,2
4,1,5,"[0.4290657440898324, -2.915033051957002, 1.910...","[-1.4572228696526315, -10.565626402356783, -9....","[-3.344741445336716, -5.2987134255227595, -21....","[-4.640225731469042, -0.6797722091697266, 4.56...","[13.213810026385428, 1.070723232433421, -11.48...","[143.26967871270682, 123.82514387564963, 106.2...","[-382.6689129130229, -337.5443524078076, -296....","[260.5524623377181, 234.08957903159518, 209.42...",...,"[1971.725146707478, 1937.538818158569, 1903.79...",22,male,1,5,300,excitement,4,4,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
409,23,14,"[0.684213500423258, 12.86551251027235, 4.33144...","[1.5698261601619976, -2.732948319527732, 9.606...","[0.9138374835292444, -4.068427179977716, -1.26...","[-0.43829591357639325, 0.04297125749970923, -3...","[-76.0124263813481, -43.028676917188115, -19.2...","[79.58811569982093, 44.62755285444659, 20.6966...","[-158.54932534523877, -145.85778241802052, -13...","[137.97281873484178, 127.77564360490732, 118.0...",...,"[187.36014580471803, 179.82246159014844, 172.4...",25,male,23,14,Gentlemans Agreement,anger,2,2,2
410,23,15,"[-0.12298562158577192, 3.4887697751389157, -0....","[0.7279736087437498, 2.9810918046378037, 4.342...","[-0.20808416856221695, -8.876289236222982, -4....","[-1.3320278365633826, 1.3158358912694685, -3.1...","[7.963247098500931, -0.912760959745539, -2.342...","[39.35946250691951, 30.910117312439496, 23.376...","[-177.48439075726597, -154.09471457492384, -13...","[179.2486334098938, 164.32618997896856, 150.24...",...,"[1115.1869452354138, 1084.399889168079, 1054.1...",25,male,23,15,Psycho,fear,2,2,2
411,23,16,"[1.2843991261938459, 15.036818145434582, 7.773...","[1.1241785467810166, -10.956

In [ ]:
final_df = merged_df.drop(["Participant", "Video"], axis=1)
final_df

,participant,video,AF3_theta,AF3_alpha,AF3_beta,AF3_gamma,AF3_IMF_1,AF3_IMF_2,AF3_IMF_3,AF3_IMF_4,...,AF4_IMF_4,AF4_IMF_5,AF4_IMF_6,Age,Gender,Video_Name,Target_Emotion,Valence,Arousal,Dominance
0,1,1,"[-1.8269399327520999, -10.694000738306702, -10...","[0.49736077657430183, 22.400426547369452, 3.30...","[1.4260946835830222, -8.421825879248049, 14.92...","[2.2820572993223274, 2.5997848143652957, -6.51...","[-8.625208967981772, -6.76122618316723, 5.8735...","[271.8306316083523, 239.358160857029, 209.7222...","[-130.83942367373743, -109.05734146942257, -89...","[-547.459771789912, -515.0312470209535, -483.9...",...,"[-12557.744211531992, -12287.397192176655, -12...","[16469.13877699183, 16128.517291506201, 15792....","[1518.4948767151295, 1518.4494358108607, 1518....",22,male,Searching for Bobby Fischer,calmness,4,3,2
1,1,2,"[0.5062581154960033, 0.912007476326554, 2.5752...","[-0.38612141446101633, -6.274040764837486, -2....","[-1.2901555098164261, -1.9465487403542245, -11...","[-2.406950438132705, -1.4401888378845544, 4.42...","[7.187299849210072, 2.48908594819667, -5.76126...","[-68.14040256449726, -56.38957483327619, -46.2...","[90.37696274362476, 77.61107707015923, 66.0474...","[5.638717004127837, 5.69588782145483, 5.703402...",...,"[741.3340755719194, 660.910481681659, 585.8071...","[-3891.0739228954917, -3828.284381573364, -376...","[-5040.771735148035, -4999.689997758618, -4958...",22,male,D.O.A.,surprise,3,3,1
2,1,3,"[-0.7417304850594953, -3.4657854661178247, -4....","[0.41278513940362516, 9.861261945260468, 2.318...","[-1.3827799882156016, -18.94342627716695, -10....","[-3.079916736719449, 3.8780490904254394, -1.73...","[-12.81640636829528, -17.285304086989484, -9.2...","[42.225517518158675, 27.278234759306788, 16.16...","[-24.0264830360964, -19.05461680680786, -14.80...","[-7.189789534072112, -5.465502664867119, -3.95...",...,"[1318.3130128562896, 1233.706411411064, 1152.8...","[-877.0838848636753, -834.441221515215, -793.1...","[-6926.798897045539, -6857.590625192434, -6788...",22,male,The Hangover,amusement,5,4,4
3,1,4,"[0.22760261879626606, 2.5935007270767776, 1.27...","[0.26103794732345525, -1.3250477622368306, 1.9...","[1.6958236158707125, 10.304412275191863, 11.12...","[2.610260469549624, -2.4266724346828616, 0.671...","[-24.84559520769245, -6.559868828735657, 0.514...","[3.967186796849267, 0.1740433577368301, -2.316...","[65.89434521625127, 55.303895894500045, 45.910...","[-96.5089931246525, -86.61348495962216, -77.40...",...,"[677.5256820346367, 619.4529066029135, 564.318...","[46.683430700235505, 43.665821605116975, 40.77...","[88.76861021840446, 85.51941443234116, 82.3323...",22,male,The Ring,fear,4,3,2
4,1,5,"[0.4290657440898324, -2.915033051957002, 1.910...","[-1.4572228696526315, -10.565626402356783, -9....","[-3.344741445336716, -5.2987134255227595, -21....","[-4.640225731469042, -0.6797722091697266, 4.56...","[13.213810026385428, 1.070723232433421, -11.48...","[143.26967871270682, 123.82514387564963, 106.2...","[-382.6689129130229, -337.5443524078076, -296....","[260.5524623377181, 234.08957903159518, 209.42...",...,"[21466.52674587031, 20412.02878445615, 19392.3...","[-3053.564197726408, -2928.516759833994, -2806...","[1971.725146707478, 1937.538818158569, 1903.79...",22,male,300,excitement,4,4,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
409,23,14,"[0.684213500423258, 12.86551251027235, 4.33144...","[1.5698261601619976, -2.732948319527732, 9.606...","[0.9138374835292444, -4.068427179977716, -1.26...","[-0.43829591357639325, 0.04297125749970923, -3...","[-76.0124263813481, -43.028676917188115, -19.2...","[79.58811569982093, 44.62755285444659, 20.6966...","[-158.54932534523877, -145.85778241802052, -13...","[137.97281873484178, 127.77564360490732, 118.0...",...,"[720.2557960779209, 677.048590766919, 635.4708...","[-771.4761325150996, -730.9156385075352, -691....","[187.36014580471803, 179.82246159014844, 172.4...",25,male,Gentlemans Agreement,anger,2,2,2
410,2

In [ ]:
final_df.to_csv("EEG_Decomposed_Features.csv", index=False)

In [440]:
final_df = pd.read_csv("EEG_Decomposed_Features.csv")
final_df

,participant,video,AF3_theta,AF3_alpha,AF3_beta,AF3_gamma,AF3_IMF_1,AF3_IMF_2,AF3_IMF_3,AF3_IMF_4,...,AF4_IMF_4,AF4_IMF_5,AF4_IMF_6,Age,Gender,Video_Name,Target_Emotion,Valence,Arousal,Dominance
0,1,1,[ -1.82693993 -10.69400074 -10.59583885 -4.03...,[ 0.49736078 22.40042655 3.30931885 10.34...,[ 1.42609468 -8.42182588 14.92431866 -18.66...,[ 2.2820573 2.59978481 -6.51163948 7.81...,[-8.62520897e+00 -6.76122618e+00 5.87357150e+...,[ 2.71830632e+02 2.39358161e+02 2.09722214e+...,[-1.30839424e+02 -1.09057341e+02 -8.94899864e+...,[-5.47459772e+02 -5.15031247e+02 -4.83919795e+...,...,[-1.25577442e+04 -1.22873972e+04 -1.20207249e+...,[ 1.64691388e+04 1.61285173e+04 1.57925872e+...,[ 1.51849488e+03 1.51844944e+03 1.51819318e+...,22,male,Searching for Bobby Fischer,calmness,4,3,2
1,1,2,[ 0.50625812 0.91200748 2.57528024 -0.93...,[-3.86121414e-01 -6.27404076e+00 -2.49647126e+...,[ -1.29015551 -1.94654874 -11.02521043 3.37...,[-2.40695044 -1.44018884 4.42969224 -6.366255...,[ 7.18729985e+00 2.48908595e+00 -5.76126377e+...,[-6.81404026e+01 -5.63895748e+01 -4.62480023e+...,[ 9.03769627e+01 7.76110771e+01 6.60474283e+...,[ 5.63871700e+00 5.69588782e+00 5.70340217e+...,...,[ 7.41334076e+02 6.60910482e+02 5.85807174e+...,[-3.89107392e+03 -3.82828438e+03 -3.76578640e+...,[-5.04077174e+03 -4.99969000e+03 -4.95866100e+...,22,male,D.O.A.,surprise,3,3,1
2,1,3,[ -0.74173049 -3.46578547 -4.37366338 -2.80...,[ 0.41278514 9.86126195 2.31878395 2.50...,[ -1.38277999 -18.94342628 -10.84841987 -19.18...,[-3.07991674 3.87804909 -1.73817031 -2.507519...,[-1.28164064e+01 -1.72853041e+01 -9.27600499e+...,[ 4.22255175e+01 2.72782348e+01 1.61691623e+...,[-2.40264830e+01 -1.90546168e+01 -1.48064626e+...,[-7.18978953e+00 -5.46550266e+00 -3.95334651e+...,...,[ 1.31831301e+03 1.23370641e+03 1.15282044e+...,[-8.77083885e+02 -8.34441222e+02 -7.93164876e+...,[-6.92679890e+03 -6.85759063e+03 -6.78866428e+...,22,male,The Hangover,amusement,5,4,4
3,1,4,[ 0.22760262 2.59350073 1.27119145 1.104799...,[ 2.61037947e-01 -1.32504776e+00 1.98620755e+...,[ 1.69582362 10.30441228 11.12802726 7.06...,[ 2.61026047 -2.42667243 0.67105551 2.666260...,[-2.48455952e+01 -6.55986883e+00 5.14723236e-...,[ 3.96718680e+00 1.74043358e-01 -2.31641402e+...,[ 6.58943452e+01 5.53038959e+01 4.59109048e+...,[-9.65089931e+01 -8.66134850e+01 -7.74047774e+...,...,[ 6.77525682e+02 6.19452907e+02 5.64318113e+...,[ 4.66834307e+01 4.36658216e+01 4.07746330e+...,[ 8.87686102e+01 8.55194144e+01 8.23323198e+...,22,male,The Ring,fear,4,3,2
4,1,5,[ 4.29065744e-01 -2.91503305e+00 1.91057423e+...,[ -1.45722287 -10.5656264 -9.05796074 0.81...,[ -3.34474145 -5.29871343 -21.96095092 5.53...,[-4.64022573 -0.67977221 4.56163175 -6.879844...,[ 13.21381003 1.07072323 -11.4885538 -7.04...,[ 1.43269679e+02 1.23825144e+02 1.06208145e+...,[-3.82668913e+02 -3.37544352e+02 -2.96076271e+...,[ 2.60552462e+02 2.34089579e+02 2.09427983e+...,...,[ 2.14665267e+04 2.04120288e+04 1.93923688e+...,[-3.05356420e+03 -2.92851676e+03 -2.80682348e+...,[ 1.97172515e+03 1.93753882e+03 1.90379910e+...,22,male,300,excitement,4,4,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
409,23,14,[ 0.6842135 12.86551251 4.33144679 9.18...,[ 1.56982616 -2.73294832 9.60624877 -14.50...,[ 9.13837484e-01 -4.06842718e+00 -1.26221167e+...,[-4.38295914e-01 4.29712575e-02 -3.31871948e+...,[-7.60124264e+01 -4.30286769e+01 -1.92029178e+...,[ 7.95881157e+01 4.46275529e+01 2.06966900e+...,[-1.58549325e+02 -1.45857782e+02 -1.33667302e+...,[ 1.37972819e+02 1.27775644e+02 1.18091735e+...,...,[ 7.20255796e+02 6.77048591e+02 6.35470893e+...,[-7.71476133e+02 -7.30915639e+02 -6.91759671e+...,[ 1.87360146e+02 1.79822462e+02 1.72470900e+...,25,male,Gentlemans Agreement,anger,2,2,2
410,23,15,[-1.22985622e-01 3.48876978e+00 -3.33545890e-...,[ 0.72797361 2.9810918 4.34204456 -2.26...,[ -0.20808417 -8.87628924 -4.50537615 -10.76...,[-1.33202784 1.31583589 -3.1902653 0.607411...,[ 7.96324710e+00 -9.12760960e-01 -2.34288472e+...,[ 3.935

## ANOVA feature extraction

In [405]:
def extract_features(signal):
    signal = np.array(signal).flatten()
    features = {}
    
    # Basic statistics
    features['Minimum'] = np.min(signal)
    features['Maximum'] = np.max(signal)
    features['Mean'] = np.mean(signal)
    features['MAV'] = np.mean(np.abs(signal))
    
    # Enhanced MAV
    p = 0.5  # example parameter for EMAV
    features['EMAV'] = np.mean(np.abs(signal)**p)
    
    # Modified MAV1 and MAV2
    n = len(signal)
    w1 = np.hamming(n)  # Example weighting function for MMAV1
    w2 = np.hanning(n)  # Example weighting function for MMAV2
    features['MMAV1'] = np.mean(np.abs(signal) * w1)
    features['MMAV2'] = np.mean(np.abs(signal) * w2)
    
    # Mean Absolute Deviation
    features['MAD'] = np.mean(np.abs(signal - np.mean(signal)))
    
    # Root Mean Square
    features['RMS'] = np.sqrt(np.mean(signal**2))
    
    # Median and Median Absolute Deviation
    features['Median'] = np.median(signal)
    features['Median_Absolute_Deviation'] = np.median(np.abs(signal - np.median(signal)))
    
    # Difference Absolute Standard Deviation Value
    diff_signal = np.diff(signal)
    features['DASDV'] = np.sqrt(np.mean(diff_signal**2))
    
    # Standard Deviation and Variance
    features['STD'] = np.std(signal)
    features['Variance'] = np.var(signal)
    
    # Coefficient Of Variance
    features['COV'] = stats.variation(signal)
    
    # Skewness and Kurtosis
    features['Skewness'] = stats.skew(signal)
    features['Kurtosis'] = stats.kurtosis(signal)
    
    # Waveform Length and Enhanced Waveform Length
    features['Waveform_Length'] = np.sum(np.abs(diff_signal))
    features['EWL'] = np.sum(np.abs(diff_signal)**p)
    
    # Interquartile Range
    features['IQR'] = stats.iqr(signal)
    
    # Average Amplitude Change
    features['AAC'] = np.mean(np.abs(diff_signal))
    
    # Log Detector
    features['Log_Detector'] = np.exp(np.mean(np.log(np.abs(signal))))
    
    # Myopulse percentage rate
    threshold = 0.1  # example threshold for MYOP
    features['MYOP'] = np.mean(np.abs(signal) > threshold)
    
    # Simple Square Integral
    features['SSI'] = np.sum(signal**2)
    
    # Willison Amplitude
    features['WA'] = np.sum(np.abs(diff_signal) > threshold)
    
    # Maximum Fractal Length
    features['MFL'] = np.sum(np.abs(np.diff(diff_signal)))
    
    # Slope Sign Change
    features['SSC'] = np.sum(np.diff(np.sign(diff_signal)) != 0)
    
    # Zero Crossing
    features['ZC'] = np.sum(np.diff(np.sign(signal)) != 0)
    
    # Entropy
    features['Entropy'] = stats.entropy(np.histogram(signal, bins=10, density=True)[0])
    
    # Shannon Entropy
    # p_signal = signal / np.sum(signal)
    bases = collections.Counter([tmp_base for tmp_base in signal])
    # define distribution
    dist = [x/sum(bases.values()) for x in bases.values()]
 
    # use scipy to calculate entropy
    entropy_value = entropy(dist, base=2)
    # features['Shannon_Entropy'] = -np.sum(p_signal * np.log2(p_signal + 1e-12))
    features['Shannon_Entropy'] = entropy_value
    # Log Energy Entropy
    features['Log_Energy_Entropy'] = np.sum(np.log2(signal**2 + 1e-12))
    
    return features


In [406]:
# Function to extract additional features
# def extract_features(signal):
#     signal = np.array(signal).flatten()
#     if len(signal) == 0:
#         raise ValueError("Signal length is zero. Cannot extract features from an empty signal.")
    
#     # Calculate histogram for entropy
#     hist, _ = np.histogram(signal, bins=10, density=True)
    # bases = collections.Counter([tmp_base for tmp_base in signal])
    # # define distribution
    # dist = [x/sum(bases.values()) for x in bases.values()]
 
    # # use scipy to calculate entropy
    # entropy_value = entropy(dist, base=2)
#     features = {
#         'Variance': np.var(signal),
#         'Mean_absolute_value': np.mean(np.abs(signal)),
#         'Mean_absolute_Deviation': np.mean(np.abs(signal - np.mean(signal))),
#         'Modified_Mean_Absolute_Value': np.mean(np.abs(signal - np.median(signal))),
#         'Modified_Mean_Absolute_Value_squared': np.mean((signal - np.median(signal))**2),
#         'Root_Mean_Square': np.sqrt(np.mean(signal**2)),
#         'Standard_Deviation': np.std(signal),
#         'Inter_Quartile_Range': np.percentile(signal, 75) - np.percentile(signal, 25),
#         'Median_Absolute_Deviation': np.median(np.abs(signal - np.median(signal))),
#         'Enhanced_Mean_Absolute_Value': np.mean(np.abs(signal)) + np.var(signal),
#         'Log_Detector': np.exp(np.mean(np.log(np.abs(signal) + 1))),
#         'Willison_Amplitude': np.sum(np.abs(np.diff(signal)) > 0.5),
#         'Myopulse_Percentage_Rate': np.sum(np.abs(signal) > 0.5 * np.max(np.abs(signal))),
#         'Entropy': entropy(hist),  # adding 1 to avoid log(0)
#         'Kurtosis': kurtosis(signal),
#         'Minimum': np.min(signal),
#         'Maximum': np.max(signal),
#         'Maximum_Fractal_Length': np.max(np.diff(signal)),
#         'Enhanced_Wavelength': np.sum(np.abs(np.diff(signal))) + np.std(signal),
#         'Difference_Absolute_STD_Value': np.std(np.diff(signal)),
#         'Log_Energy_Entropy': np.sum(signal**2 * np.log(signal**2 + 1e-9)),
#         'Average_Amplitude_Change': np.mean(np.abs(np.diff(signal))),
#         'Zero_Crossing': np.sum(np.diff(np.sign(signal)) != 0),
#         'Wavelength': np.sum(np.abs(np.diff(signal))),
#         'Slope_Sign_Change': np.sum(np.diff(np.sign(np.diff(signal))) != 0),
#         'Simple_Square_Integral': np.sum(np.array(signal).flatten()**2),
#         'Skewness': skew(signal),
#         'Mean': np.mean(signal),
#         'Covariance': np.cov(signal).item() if len(signal) > 1 else 0,  # handle case where length is 1
#         'Median': np.median(signal),
#         'Shannon_Entropy': entropy_value,
#     }
#     return features

In [407]:
def feature_extraction_ANOVA_EEG(raw, secs):
    # 128 Hz is the sampling rate for the EEG data
    fs_EEG = 128
    N_EEG = math.ceil(fs_EEG * secs)
    features = []
    # for electrode in eeg_electrodes:
    #     features += [f'{electrode}_theta', f'{electrode}_alpha', f'{electrode}_beta', f'{electrode}_gamma'] + [f'{electrode}_IMF_{i + 1}' for i in range(6)]
    
    additional_features = [
        'Minimum', 'Maximum', 'Mean', 'MAV', 'EMAV',
        'MMAV1', 'MMAV2', 'MAD', 'RMS', 'Median',
        'Median_Absolute_Deviation', 'DASDV', 'STD', 'Variance', 'COV',
        'Skewness', 'Kurtosis', 'Waveform_Length', 'EWL', 'IQR',
        'AAC', 'Log_Detector', 'MYOP', 'SSI', 'WA',
        'MFL', 'SSC', 'ZC', 'Entropy', 'Shannon_Entropy',
        'Log_Energy_Entropy'
    ]

    # Adding additional feature columns for each electrode
    for electrode in eeg_electrodes:
        features += [f'{electrode}_{feat}' for feat in additional_features]
    
    # DataFrame to store features
    columns = ['participant', 'video'] + features + ['Valence', 'Arousal', 'Target_Emotion']
    df = pd.DataFrame(columns=columns)
    
    for participant in range(23):
        for video in range(18):
            feature_values = []
            for i in range(14):
                # basl = raw['DREAMER'][0, 0]['Data'][0, participant]['EEG'][0, 0]['baseline'][0, 0][video, 0][-1 - N_EEG:-1, i]
                stimuli = raw['DREAMER'][0, 0]['Data'][0, participant]['EEG'][0, 0]['stimuli'][0, 0][video, 0][-1 - N_EEG:-1, i]
                # Detrending
                # basl_detrended = detrend_signal(basl)
                # stimuli_detrended = detrend_signal(stimuli)

                # Baseline correction
                # stimuli_corrected = baseline_correction(basl_detrended, stimuli_detrended)

                # Feature extraction
                # DWT_stimuli = dwt_and_band_extraction([], stimuli_corrected)
                # emd_stimuli = emd(stimuli_corrected)

                # Combine features
                # feature_values += DWT_stimuli + emd_stimuli

                # print(stimuli_corrected.shape)
                # print(stimuli_corrected)
                # Additional feature extraction
                additional_features = extract_features([stimuli])
                feature_values += additional_features.values()

            # Assuming Valence and Arousal are stored similarly to the EEG signals
            valence = raw['DREAMER'][0, 0]['Data'][0, participant]['ScoreValence'][0, 0][video, 0]
            arousal = raw['DREAMER'][0, 0]['Data'][0, participant]['ScoreArousal'][0, 0][video, 0]
            target_emotion = ['calmness', 'surprise', 'amusement', 'fear', 'excitement', 'disgust',
                  'happiness', 'anger', 'sadness', 'disgust', 'calmness', 'amusement',
                  'happiness', 'anger', 'fear', 'excitement', 'sadness', 'surprise'][video]

            # Append to feature list
            row = [participant + 1, video + 1] + feature_values + [valence, arousal, target_emotion]
            if len(row) == len(columns):
                df.loc[len(df)] = row
            else:
                print(f'Skipping participant {participant + 1}, video {video + 1}')
                print(len(row))
                print(row)
                print(len(columns))
                print(columns)

    return df

In [408]:
# Assuming `raw` is the dataset loaded and `secs` is the time duration for which features are extracted
secs = 5
features_df = feature_extraction_ANOVA_EEG(raw, secs)

In [427]:
features_df

,participant,video,AF3_Minimum,AF3_Maximum,AF3_Mean,AF3_MAV,AF3_EMAV,AF3_MMAV1,AF3_MMAV2,AF3_MAD,...,AF4_WA,AF4_MFL,AF4_SSC,AF4_ZC,AF4_Entropy,AF4_Shannon_Entropy,AF4_Log_Energy_Entropy,Valence,Arousal,Target_Emotion
0,1,1,4361.025641,4397.435897,4379.935897,4379.935897,66.181058,2362.131654,2186.670416,6.726563,...,635,25163.589744,281,0,1.897167,8.265993,15395.785929,4,3,calmness
1,1,2,4371.282051,4401.538462,4387.022436,4387.022436,66.234578,2365.970899,2190.227287,6.583433,...,633,18401.025641,279,0,1.990009,8.262816,15406.951696,3,3,surprise
2,1,3,4372.820513,4403.589744,4388.475160,4388.475160,66.245547,2366.841232,2191.046977,6.105812,...,636,16928.717949,274,0,1.909144,8.250871,15402.177228,5,4,amusement
3,1,4,4376.410256,4407.692308,4392.376603,4392.376603,66.274988,2368.728853,2192.759483,5.798387,...,636,33108.205128,281,0,1.868232,8.212605,15400.228231,4,3,fear
4,1,5,4372.820513,4401.538462,4388.656250,4388.656250,66.246911,2366.732169,2190.912683,6.358439,...,638,66443.076923,280,0,2.110378,8.556350,15408.018057,4,4,excitement
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
409,23,14,4355.384615,4420.512821,4388.595353,4388.595353,66.246403,2365.926527,2190.042282,10.446211,...,611,4213.846154,288,0,1.738233,6.694237,15402.246439,2,2,anger
410,23,15,4341.538462,4431.282051,4396.370994,4396.370994,66.305028,2373.286133,2197.365711,12.288880,...,618,3765.641026,283,0,1.910083,6.683362,15401.362109,2,2,fear
411,23,16,4353.846154,4450.256410,4389.412660,4389.412660,66.252537,2368.209347,2192.452537,12.171487,...,603,2774.871795,285,0,2.068464,6.663696,15395.872949,3,3,excitement
412,23,17,4367.179487,4427.692308,4399.836538,4399.836538,66.331209,2371.809291,2195.459096,8.730469,...,610,3588.205128,316,0,1.268438,6.537706,15404.785302,2,2,sadness


In [428]:
additional_features = [
    'Minimum', 'Maximum', 'Mean', 'MAV', 'EMAV',
    'MMAV1', 'MMAV2', 'MAD', 'RMS', 'Median',
    'Median_Absolute_Deviation', 'DASDV', 'STD', 'Variance', 'COV',
    'Skewness', 'Kurtosis', 'Waveform_Length', 'EWL', 'IQR',
    'AAC', 'Log_Detector', 'MYOP', 'SSI', 'WA',
    'MFL', 'SSC', 'ZC', 'Entropy', 'Shannon_Entropy',
    'Log_Energy_Entropy'
]

features = []
# Adding additional feature columns for each electrode
for electrode in eeg_electrodes:
    features += [f'{electrode}_{feat}' for feat in additional_features]

In [429]:
cnt = 0
for feat in features:
    if features_df[feat].dtype == 'object':
        print(feat)
        cnt+=1
print(cnt)

0


In [430]:
features_df['AF4_Shannon_Entropy']

0      8.265993
1      8.262816
2      8.250871
3      8.212605
4      8.556350
         ...   
409    6.694237
410    6.683362
411    6.663696
412    6.537706
413    6.720958
Name: AF4_Shannon_Entropy, Length: 414, dtype: float64

In [431]:
# Split the DataFrame for arousal and valence separately
features_arousal_df = features_df.drop(columns=['Valence']).copy()
features_valence_df = features_df.drop(columns=['Arousal']).copy()
# Replace Inf and -Inf with NaN
features_arousal_df.replace([np.inf, -np.inf], np.nan, inplace=True)
# Drop rows with NaN values
features_arousal_df.dropna(inplace=True)
# Replace Inf and -Inf with NaN
features_valence_df.replace([np.inf, -np.inf], np.nan, inplace=True)
# Drop rows with NaN values
features_valence_df.dropna(inplace=True)
# Remove constant features
constant_filter = VarianceThreshold(threshold=0)
X_arousal = constant_filter.fit_transform(features_arousal_df.drop(columns=['participant', 'video', 'Target_Emotion']))
# Update feature names after removing constant features
filtered_feature_names = features_arousal_df.drop(columns=['participant', 'video', 'Target_Emotion']).columns[constant_filter.get_support()]

In [432]:
filtered_feature_names

Index(['AF3_Minimum', 'AF3_Maximum', 'AF3_Mean', 'AF3_MAV', 'AF3_EMAV',
       'AF3_MMAV1', 'AF3_MMAV2', 'AF3_MAD', 'AF3_RMS', 'AF3_Median',
       ...
       'AF4_AAC', 'AF4_Log_Detector', 'AF4_SSI', 'AF4_WA', 'AF4_MFL',
       'AF4_SSC', 'AF4_Entropy', 'AF4_Shannon_Entropy',
       'AF4_Log_Energy_Entropy', 'Arousal'],
      dtype='object', length=407)

In [433]:
# Original feature names before removing constant features
original_feature_names = features_arousal_df.drop(columns=['participant', 'video', 'Target_Emotion']).columns

# Mask indicating constant features
constant_mask = np.logical_not(constant_filter.get_support())

# Get the names of constant features
constant_features = original_feature_names[constant_mask]

print("Constant features in X_arousal:")
print(constant_features)
# Convert X_arousal back to DataFrame with updated feature names
X_arousal_df = pd.DataFrame(X_arousal, columns=filtered_feature_names)
# Define the target variable
y_arousal = features_arousal_df['Target_Emotion']

# Perform ANOVA
anova_results_arousal = f_classif(X_arousal_df, y_arousal)
f_values_arousal, p_values_arousal = anova_results_arousal

# Create a DataFrame to hold the results
anova_arousal_df = pd.DataFrame({
    'Feature': filtered_feature_names,
    'F-value': f_values_arousal,
    'p-value': p_values_arousal
})

# Sort the DataFrame by p-value
anova_arousal_df.sort_values(by='p-value', inplace=True)

# Select significant features with p-value < 0.05
significant_features_arousal = anova_arousal_df[anova_arousal_df['p-value'] < 0.05]

Constant features in X_arousal:
Index(['AF3_MYOP', 'AF3_ZC', 'F7_MYOP', 'F7_ZC', 'F3_MYOP', 'F3_ZC',
       'FC5_MYOP', 'FC5_ZC', 'T7_MYOP', 'T7_ZC', 'P7_MYOP', 'P7_ZC', 'O1_MYOP',
       'O1_ZC', 'O2_MYOP', 'O2_ZC', 'P8_MYOP', 'P8_ZC', 'T8_MYOP', 'T8_ZC',
       'FC6_MYOP', 'FC6_ZC', 'F4_MYOP', 'F4_ZC', 'F8_MYOP', 'F8_ZC',
       'AF4_MYOP', 'AF4_ZC'],
      dtype='object')


In [434]:
anova_arousal_df

,Feature,F-value,p-value
406,Arousal,10.599937,1.516647e-13
151,P7_MMAV2,3.412888,8.232524e-04
150,P7_MMAV1,3.391235,8.783128e-04
154,P7_Median,3.321803,1.080434e-03
167,P7_SSI,3.141545,1.843044e-03
...,...,...,...
49,F7_AAC,0.027129,9.999946e-01
53,F7_MFL,0.026788,9.999948e-01
11,AF3_DASDV,0.025463,9.999957e-01
47,F7_EWL,0.021074,9.999980e-01


In [441]:
significant_features_arousal

,Feature,F-value,p-value
406,Arousal,10.599937,1.516647e-13
151,P7_MMAV2,3.412888,8.232524e-04
150,P7_MMAV1,3.391235,8.783128e-04
154,P7_Median,3.321803,1.080434e-03
167,P7_SSI,3.141545,1.843044e-03
153,P7_RMS,3.136982,1.867990e-03
148,P7_MAV,3.122628,1.948651e-03
147,P7_Mean,3.122628,1.948651e-03
149,P7_EMAV,3.112313,2.008707e-03
166,P7_Log_Detector,3.107726,2.035992e-03


In [442]:
constant_filter = VarianceThreshold(threshold=0)
X_valence = constant_filter.fit_transform(features_valence_df.drop(columns=['participant', 'video', 'Target_Emotion']))
# Update feature names after removing constant features
filtered_feature_names = features_valence_df.drop(columns=['participant', 'video', 'Target_Emotion']).columns[constant_filter.get_support()]
# Perform ANOVA for valence
X_valence_df = pd.DataFrame(X_valence, columns=filtered_feature_names)
y_valence = features_valence_df['Target_Emotion']

anova_results_valence = f_classif(X_valence_df, y_valence)
f_values_valence, p_values_valence = anova_results_valence

anova_valence_df = pd.DataFrame({
    'Feature': X_valence_df.columns,
    'F-value': f_values_valence,
    'p-value': p_values_valence
})
anova_valence_df.sort_values(by='p-value', inplace=True)
significant_features_valence = anova_valence_df[anova_valence_df['p-value'] < 0.05]

# The significant features are stored in `significant_features_arousal` and `significant_features_valence`

In [443]:
anova_valence_df

,Feature,F-value,p-value
406,Valence,65.173755,4.404425e-68
151,P7_MMAV2,3.412888,8.232524e-04
150,P7_MMAV1,3.391235,8.783128e-04
154,P7_Median,3.321803,1.080434e-03
167,P7_SSI,3.141545,1.843044e-03
...,...,...,...
49,F7_AAC,0.027129,9.999946e-01
53,F7_MFL,0.026788,9.999948e-01
11,AF3_DASDV,0.025463,9.999957e-01
47,F7_EWL,0.021074,9.999980e-01


In [444]:
significant_features_valence

,Feature,F-value,p-value
406,Valence,65.173755,4.404425e-68
151,P7_MMAV2,3.412888,8.232524e-04
150,P7_MMAV1,3.391235,8.783128e-04
154,P7_Median,3.321803,1.080434e-03
167,P7_SSI,3.141545,1.843044e-03
153,P7_RMS,3.136982,1.867990e-03
148,P7_MAV,3.122628,1.948651e-03
147,P7_Mean,3.122628,1.948651e-03
149,P7_EMAV,3.112313,2.008707e-03
166,P7_Log_Detector,3.107726,2.035992e-03


In [422]:
valence_columns_to_add = list(significant_features_valence['Feature'])
# Check if final_df already has these columns
existing_columns_valence = final_df.columns.intersection(valence_columns_to_add)
new_columns_valence = [col for col in valence_columns_to_add if col not in existing_columns_valence]
final_df = pd.concat([final_df, features_valence_df[new_columns_valence]], axis=1)

In [423]:
final_df

,participant,video,AF3_theta,AF3_alpha,AF3_beta,AF3_gamma,AF3_IMF_1,AF3_IMF_2,AF3_IMF_3,AF3_IMF_4,...,P7_MAV,P7_Mean,P7_EMAV,P7_Log_Detector,P7_Log_Energy_Entropy,F8_SSC,F8_Median,T8_WA,T8_SSC,P8_Skewness
0,1,1,[ -1.82693993 -10.69400074 -10.59583885 -4.03...,[ 0.49736078 22.40042655 3.30931885 10.34...,[ 1.42609468 -8.42182588 14.92431866 -18.66...,[ 2.2820573 2.59978481 -6.51163948 7.81...,[-8.62520897e+00 -6.76122618e+00 5.87357150e+...,[ 2.71830632e+02 2.39358161e+02 2.09722214e+...,[-1.30839424e+02 -1.09057341e+02 -8.94899864e+...,[-5.47459772e+02 -5.15031247e+02 -4.83919795e+...,...,4367.303686,4367.303686,66.085474,4367.275983,15478.423024,279,4297.692308,636,282,0.245401
1,1,2,[ 0.50625812 0.91200748 2.57528024 -0.93...,[-3.86121414e-01 -6.27404076e+00 -2.49647126e+...,[ -1.29015551 -1.94654874 -11.02521043 3.37...,[-2.40695044 -1.44018884 4.42969224 -6.366255...,[ 7.18729985e+00 2.48908595e+00 -5.76126377e+...,[-6.81404026e+01 -5.63895748e+01 -4.62480023e+...,[ 9.03769627e+01 7.76110771e+01 6.60474283e+...,[ 5.63871700e+00 5.69588782e+00 5.70340217e+...,...,4353.049679,4353.049679,65.977556,4353.026122,15472.387792,276,4301.282051,637,280,-0.352233
2,1,3,[ -0.74173049 -3.46578547 -4.37366338 -2.80...,[ 0.41278514 9.86126195 2.31878395 2.50...,[ -1.38277999 -18.94342628 -10.84841987 -19.18...,[-3.07991674 3.87804909 -1.73817031 -2.507519...,[-1.28164064e+01 -1.72853041e+01 -9.27600499e+...,[ 4.22255175e+01 2.72782348e+01 1.61691623e+...,[-2.40264830e+01 -1.90546168e+01 -1.48064626e+...,[-7.18978953e+00 -5.46550266e+00 -3.95334651e+...,...,4335.223558,4335.223558,65.842347,4335.205786,15464.812496,274,4336.153846,635,278,-0.775921
3,1,4,[ 0.22760262 2.59350073 1.27119145 1.104799...,[ 2.61037947e-01 -1.32504776e+00 1.98620755e+...,[ 1.69582362 10.30441228 11.12802726 7.06...,[ 2.61026047 -2.42667243 0.67105551 2.666260...,[-2.48455952e+01 -6.55986883e+00 5.14723236e-...,[ 3.96718680e+00 1.74043358e-01 -2.31641402e+...,[ 6.58943452e+01 5.53038959e+01 4.59109048e+...,[-9.65089931e+01 -8.66134850e+01 -7.74047774e+...,...,4349.116186,4349.116186,65.947776,4349.102008,15470.722345,280,4290.769231,639,278,-0.046095
4,1,5,[ 4.29065744e-01 -2.91503305e+00 1.91057423e+...,[ -1.45722287 -10.5656264 -9.05796074 0.81...,[ -3.34474145 -5.29871343 -21.96095092 5.53...,[-4.64022573 -0.67977221 4.56163175 -6.879844...,[ 13.21381003 1.07072323 -11.4885538 -7.04...,[ 1.43269679e+02 1.23825144e+02 1.06208145e+...,[-3.82668913e+02 -3.37544352e+02 -2.96076271e+...,[ 2.60552462e+02 2.34089579e+02 2.09427983e+...,...,4349.614583,4349.614583,65.951555,4349.600646,15470.934057,280,4312.051282,638,280,0.152249
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
409,23,14,[ 0.6842135 12.86551251 4.33144679 9.18...,[ 1.56982616 -2.73294832 9.60624877 -14.50...,[ 9.13837484e-01 -4.06842718e+00 -1.26221167e+...,[-4.38295914e-01 4.29712575e-02 -3.31871948e+...,[-7.60124264e+01 -4.30286769e+01 -1.92029178e+...,[ 7.95881157e+01 4.46275529e+01 2.06966900e+...,[-1.58549325e+02 -1.45857782e+02 -1.33667302e+...,[ 1.37972819e+02 1.27775644e+02 1.18091735e+...,...,4374.291667,4374.291667,66.138357,4374.272777,15481.379166,293,4309.230769,608,290,-0.251621
410,23,15,[-1.22985622e-01 3.48876978e+00 -3.33545890e-...,[ 0.72797361 2.9810918 4.34204456 -2.26...,[ -0.20808417 -8.87628924 -4.50537615 -10.76...,[-1.33202784 1.31583589 -3.1902653 0.607411...,[ 7.96324710e+00 -9.12760960e-01 -2.34288472e+...,[ 3.93594625e+01 3.09101173e+01 2.33767774e+...,[-1.77484391e+02 -1.54094715e+02 -1.33192338e+...,[ 1.79248633e+02 1.64326190e+02 1.50249877e+...,...,4410.947115,4410.947115,66.414651,4410.864497,15496.762529,294,4308.717949,612,297,0.029809
411,23,16,[ 1.28439913 15.03681815 7.77309418 6.04...,[ 1.12417855 -10.95644981 7.31647702 -11.95...,[-4.03933170e-01 -4.93472163e+00 -1.41369727e+...,[-2.88710485 -0.505177 4.6319135 -4.830137...,[-7.51958273e-01 -3.79879697e+00 -8.74356314e+...,[ 8.01455078e+00 5.56963517e+00 3.29021895e+..

In [426]:
final_df.to_csv("Final_Valence_Features.csv", index=False)

In [445]:
arousal_columns_to_add = list(significant_features_arousal['Feature'])
existing_columns_arousal = final_df.columns.intersection(arousal_columns_to_add)
new_columns_arousal = [col for col in arousal_columns_to_add if col not in existing_columns_arousal]
final_df = pd.concat([final_df, features_arousal_df[new_columns_arousal]], axis=1)

In [446]:
final_df

,participant,video,AF3_theta,AF3_alpha,AF3_beta,AF3_gamma,AF3_IMF_1,AF3_IMF_2,AF3_IMF_3,AF3_IMF_4,...,P7_MAV,P7_Mean,P7_EMAV,P7_Log_Detector,P7_Log_Energy_Entropy,F8_SSC,F8_Median,T8_WA,T8_SSC,P8_Skewness
0,1,1,[ -1.82693993 -10.69400074 -10.59583885 -4.03...,[ 0.49736078 22.40042655 3.30931885 10.34...,[ 1.42609468 -8.42182588 14.92431866 -18.66...,[ 2.2820573 2.59978481 -6.51163948 7.81...,[-8.62520897e+00 -6.76122618e+00 5.87357150e+...,[ 2.71830632e+02 2.39358161e+02 2.09722214e+...,[-1.30839424e+02 -1.09057341e+02 -8.94899864e+...,[-5.47459772e+02 -5.15031247e+02 -4.83919795e+...,...,4367.303686,4367.303686,66.085474,4367.275983,15478.423024,279,4297.692308,636,282,0.245401
1,1,2,[ 0.50625812 0.91200748 2.57528024 -0.93...,[-3.86121414e-01 -6.27404076e+00 -2.49647126e+...,[ -1.29015551 -1.94654874 -11.02521043 3.37...,[-2.40695044 -1.44018884 4.42969224 -6.366255...,[ 7.18729985e+00 2.48908595e+00 -5.76126377e+...,[-6.81404026e+01 -5.63895748e+01 -4.62480023e+...,[ 9.03769627e+01 7.76110771e+01 6.60474283e+...,[ 5.63871700e+00 5.69588782e+00 5.70340217e+...,...,4353.049679,4353.049679,65.977556,4353.026122,15472.387792,276,4301.282051,637,280,-0.352233
2,1,3,[ -0.74173049 -3.46578547 -4.37366338 -2.80...,[ 0.41278514 9.86126195 2.31878395 2.50...,[ -1.38277999 -18.94342628 -10.84841987 -19.18...,[-3.07991674 3.87804909 -1.73817031 -2.507519...,[-1.28164064e+01 -1.72853041e+01 -9.27600499e+...,[ 4.22255175e+01 2.72782348e+01 1.61691623e+...,[-2.40264830e+01 -1.90546168e+01 -1.48064626e+...,[-7.18978953e+00 -5.46550266e+00 -3.95334651e+...,...,4335.223558,4335.223558,65.842347,4335.205786,15464.812496,274,4336.153846,635,278,-0.775921
3,1,4,[ 0.22760262 2.59350073 1.27119145 1.104799...,[ 2.61037947e-01 -1.32504776e+00 1.98620755e+...,[ 1.69582362 10.30441228 11.12802726 7.06...,[ 2.61026047 -2.42667243 0.67105551 2.666260...,[-2.48455952e+01 -6.55986883e+00 5.14723236e-...,[ 3.96718680e+00 1.74043358e-01 -2.31641402e+...,[ 6.58943452e+01 5.53038959e+01 4.59109048e+...,[-9.65089931e+01 -8.66134850e+01 -7.74047774e+...,...,4349.116186,4349.116186,65.947776,4349.102008,15470.722345,280,4290.769231,639,278,-0.046095
4,1,5,[ 4.29065744e-01 -2.91503305e+00 1.91057423e+...,[ -1.45722287 -10.5656264 -9.05796074 0.81...,[ -3.34474145 -5.29871343 -21.96095092 5.53...,[-4.64022573 -0.67977221 4.56163175 -6.879844...,[ 13.21381003 1.07072323 -11.4885538 -7.04...,[ 1.43269679e+02 1.23825144e+02 1.06208145e+...,[-3.82668913e+02 -3.37544352e+02 -2.96076271e+...,[ 2.60552462e+02 2.34089579e+02 2.09427983e+...,...,4349.614583,4349.614583,65.951555,4349.600646,15470.934057,280,4312.051282,638,280,0.152249
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
409,23,14,[ 0.6842135 12.86551251 4.33144679 9.18...,[ 1.56982616 -2.73294832 9.60624877 -14.50...,[ 9.13837484e-01 -4.06842718e+00 -1.26221167e+...,[-4.38295914e-01 4.29712575e-02 -3.31871948e+...,[-7.60124264e+01 -4.30286769e+01 -1.92029178e+...,[ 7.95881157e+01 4.46275529e+01 2.06966900e+...,[-1.58549325e+02 -1.45857782e+02 -1.33667302e+...,[ 1.37972819e+02 1.27775644e+02 1.18091735e+...,...,4374.291667,4374.291667,66.138357,4374.272777,15481.379166,293,4309.230769,608,290,-0.251621
410,23,15,[-1.22985622e-01 3.48876978e+00 -3.33545890e-...,[ 0.72797361 2.9810918 4.34204456 -2.26...,[ -0.20808417 -8.87628924 -4.50537615 -10.76...,[-1.33202784 1.31583589 -3.1902653 0.607411...,[ 7.96324710e+00 -9.12760960e-01 -2.34288472e+...,[ 3.93594625e+01 3.09101173e+01 2.33767774e+...,[-1.77484391e+02 -1.54094715e+02 -1.33192338e+...,[ 1.79248633e+02 1.64326190e+02 1.50249877e+...,...,4410.947115,4410.947115,66.414651,4410.864497,15496.762529,294,4308.717949,612,297,0.029809
411,23,16,[ 1.28439913 15.03681815 7.77309418 6.04...,[ 1.12417855 -10.95644981 7.31647702 -11.95...,[-4.03933170e-01 -4.93472163e+00 -1.41369727e+...,[-2.88710485 -0.505177 4.6319135 -4.830137...,[-7.51958273e-01 -3.79879697e+00 -8.74356314e+...,[ 8.01455078e+00 5.56963517e+00 3.29021895e+..

In [447]:
final_df.to_csv("Final_Arousal_Features.csv", index=False)